In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

model = models.Sequential([
layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
layers.MaxPooling2D((2,2)),
layers.Conv2D(64, (3,3), activation='relu'),
layers.MaxPooling2D((2,2)),
layers.Flatten(),
layers.Dense(64, activation='relu'),
layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


model.save('mnist_model.h5')
print("Mô hình đã được lưu: mnist_model.h5")

/Users/minhtuansfile/HoangTuan_Code/XLA_BTL/venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9571 - loss: 0.1421 - val_accuracy: 0.9835 - val_loss: 0.0514
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9853 - loss: 0.0479 - val_accuracy: 0.9889 - val_loss: 0.0337
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9895 - loss: 0.0334 - val_accuracy: 0.9826 - val_loss: 0.0499
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9923 - loss: 0.0242 - val_accuracy: 0.9893 - val_loss: 0.0316
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9942 - loss: 0.0186 - val_accuracy: 0.9899 - val_loss: 0.0283


✅ Mô hình đã được lưu: mnist_model.h5


In [11]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.legend()
plt.title('Độ chính xác của mô hình CNN trên MNIST')
plt.show()

<Figure size 640x480 with 1 Axes>

In [ ]:
import gradio as gr
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from PIL import Image
import json
import os

try:
    model = load_model('mnist_model.h5', compile=False)
except IOError:
    print("LỖI: Không tìm thấy file 'mnist_model.h5'. Hãy đảm bảo bạn đã đặt file model vào đúng thư mục.")

def predict_image(img):
    try:
        if img is None:
            return "Vui lòng vẽ một số trước."
        if isinstance(img, str):
            try:
                img = json.loads(img)
            except json.JSONDecodeError:
                return "Dữ liệu Flag không hợp lệ (không phải JSON)."
        if isinstance(img, dict):
            if "composite" in img and img["composite"] is not None:
                img_data = img["composite"]

                if isinstance(img_data, str):
                    if os.path.exists(img_data):
                        img = cv2.imread(img_data, cv2.IMREAD_UNCHANGED)
                    else:
                        return f"Không tìm thấy file ảnh (Flag): {img_data}"
                elif isinstance(img_data, np.ndarray):
                    img = img_data  
                else:
                    return f"Dữ liệu 'composite' không xác định: {type(img_data)}"
            else:
                return "Không có ảnh hợp lệ trong dữ liệu Flag."
        if isinstance(img, Image.Image):
            img = np.array(img)
        if not isinstance(img, np.ndarray):
            return f"Dữ liệu ảnh không xác định: {type(img)}"
        if len(img.shape) == 3:
            if img.shape[-1] == 4:
                img_gray = cv2.cvtColor(img, cv2.COLOR_RGBA2GRAY)
            elif img.shape[-1] == 3:
                img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            else:
                img_gray = img[:,:,0] 
        else:
            img_gray = img

        img_resized = cv2.resize(img_gray, (28, 28))

        img_inverted = cv2.bitwise_not(img_resized)

        img_norm = img_inverted / 255.0
        img_input = img_norm.reshape(1, 28, 28, 1)

        pred = model.predict(img_input)
        result = np.argmax(pred)
        conf = np.max(pred)
        return f"🧠 Mô hình dự đoán: {result} (độ tin cậy: {conf:.2f})"

    except Exception as e:
        import traceback
        print(traceback.format_exc())
        return f"Lỗi xử lý ảnh: {str(e)}"

interface = gr.Interface(
    fn=predict_image,

    inputs=gr.Sketchpad(label="Vẽ số của bạn", type="numpy"), 
    
    outputs=gr.Textbox(label="Kết quả dự đoán"),
    title="Nhận dạng chữ số viết tay (MNIST)",
    description="Vẽ một số (0–9) bằng chuột, sau đó nhấn Submit để mô hình dự đoán.",
)

# 4. Chạy
interface.launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
